In [1]:
public enum Method { Linear = 0, TimeRefs = 1, OutterForce = 2 }

In [2]:
public class MalthusianGrowthModel
{
    public double A { get; private set;}
    public double B { get; private set;}
    public double Y { get; private set;}
    public int N0 { get; private set;}
    public double T0 { get; private set;}
    public Func<double, double, double> OutterForce { get; private set;}

    public double R => A - B;
    
    public MalthusianGrowthModel WithA(double a) { A = a; return this; }
    public MalthusianGrowthModel WithB(double b) { B = b; return this; }
    public MalthusianGrowthModel WithY(double y) { Y = y; return this; }
    public MalthusianGrowthModel WithN0(int n0) { N0 = n0; prev = n0; return this; }
    public MalthusianGrowthModel WithT0(double t0) { T0 = t0; return this; }
    public MalthusianGrowthModel WithOutter(Func<double, double, double> outter) { OutterForce = outter; return this; }

    private double prev;
    public IEnumerable<double> Run(double timeStep, double timeLimit, Method f)
    {
        Func<double, double, double> func = f switch
        {
            Method.Linear => GenerateFirst,
            Method.TimeRefs => GenerateSecond,
            Method.OutterForce => GenerateThird,
            _ => null
        };

        var values = new List<double>();
        for (double i = T0; i < timeLimit; i += timeStep)
        {            
            values.Add(Math.Round(func(i, timeStep), 2));
        }

        return values;
    }

    private double GenerateFirst(double t, double step)
    {
        return N0 * Math.Exp((A - B) * (t - T0));
    }

    private double GenerateSecond(double t, double step)
    {
        return (Y * N0 * Math.Exp(R * (t - T0))) / (Y + N0 * (Math.Exp(R * (t - T0)) - 1));
    }

    public double GenerateThird(double t, double step)
    {
        prev = prev + step * (R * prev * (1 - prev / (Y * OutterForce(prev, t))));
        return prev;
    }
}

In [3]:
#r "nuget:XPlot.Plotly"
#r "nuget:Microsoft.Data.Analysis"
#r "nuget:XPlot.Plotly.Interactive"

Installed Packages Microsoft.Data.Analysis, 0.20.1 XPlot.Plotly, 4.0.6 XPlot.Plotly.Interactive, 4.0.6

Loading extensions from `/Users/mjajksj/.nuget/packages/xplot.plotly.interactive/4.0.6/interactive-extensions/dotnet/XPlot.Plotly.Interactive.dll`

Configuring PowerShell Kernel for XPlot.Plotly integration.

Installed support for XPlot.Plotly.

Loading extensions from `/Users/mjajksj/.nuget/packages/microsoft.data.analysis/0.20.1/interactive-extensions/dotnet/Microsoft.Data.Analysis.Interactive.dll`

In [4]:
using XPlot.Plotly;
using Microsoft.Data.Analysis;
using System.Linq;
using XPlot.Plotly.Interactive;

In [5]:
var timeLimit = 10.0;
var timeStep = 1.0;
var timeStart = 0.0;
var people = 200;
var time = new List<double>();
for (double i = timeStart; i < timeLimit; i += timeStep) { time.Add(i); }

var layout = new Layout.Layout 
{
    xaxis = new Xaxis 
    {
        title = $"Время, шаг: {timeStep}",
    },
    yaxis = new Yaxis
    {
        title = "Популяция"
    },
    title = $"Модель роста популяции"
};

In [6]:
var first1 = new MalthusianGrowthModel()
    .WithN0(people)
    .WithT0(timeStart)
    .WithA(0.6)
    .WithB(0.5)
    .Run(timeStep, timeLimit, Method.Linear);

var first2 = new MalthusianGrowthModel()
    .WithN0(people)
    .WithT0(timeStart)
    .WithA(0.5)
    .WithB(0.6)
    .Run(timeStep, timeLimit, Method.Linear);

var first3 = new MalthusianGrowthModel()
    .WithN0(people)
    .WithT0(timeStart)
    .WithA(0.5)
    .WithB(0.5)
    .Run(timeStep, timeLimit, Method.Linear);

var firstScatter1 = new Scatter
{   
    x = time,
    y = first1,
    marker = new Marker { color = "red" },
    line = new Line { shape = "spline" }
};

var firstScatter2 = new Scatter
{   
    x = time,
    y = first2,
    marker = new Marker { color = "blue" },
    line = new Line { shape = "spline" }
};

var firstScatter3 = new Scatter
{   
    x = time,
    y = first3,
    marker = new Marker { color = "green" },
    line = new Line { shape = "spline" }
};

Chart.Plot(new List<Trace>() { firstScatter1, firstScatter2, firstScatter3 }, layout)

<!DOCTYPE html>

In [7]:
var second1 = new MalthusianGrowthModel()
    .WithN0(people)
    .WithT0(timeStart)
    .WithA(10)
    .WithB(9)
    .WithY(500)
    .Run(timeStep, timeLimit, Method.TimeRefs);

var second2 = new MalthusianGrowthModel()
    .WithN0(4 * people)
    .WithT0(timeStart)
    .WithA(10)
    .WithB(8)
    .WithY(500)
    .Run(timeStep, timeLimit, Method.TimeRefs);

var secondScatter1 = new Scatter
{   
    x = time,
    y = second1,
    marker = new Marker { color = "blue" },
    line = new Line { shape = "spline" }
};

var secondScatter2 = new Scatter
{   
    x = time,
    y = second2,
    marker = new Marker { color = "green" },
    line = new Line { shape = "spline" }
};

Chart.Plot(new List<Trace>() { secondScatter1, secondScatter2 }, layout)

<!DOCTYPE html>

In [8]:
var timeLimit = 60.0;
var time = new List<double>();
for (double i = timeStart; i < timeLimit; i += timeStep) { time.Add(i); }

var third1 = new MalthusianGrowthModel()
    .WithN0(people)
    .WithT0(timeStart)
    .WithA(10)
    .WithB(9)
    .WithY(500.0)
    .WithOutter((prev, t) => 1)
    .Run(timeStep, timeLimit, Method.OutterForce);

var third2 = new MalthusianGrowthModel()
    .WithN0(people)
    .WithT0(timeStart)
    .WithA(10)
    .WithB(9)
    .WithY(500.0)
    .WithOutter((prev, t) => 2)
    .Run(timeStep, timeLimit, Method.OutterForce);

var third3 = new MalthusianGrowthModel()
    .WithN0(people)
    .WithT0(timeStart)
    .WithA(10)
    .WithB(9)
    .WithY(500.0)
    .WithOutter((prev, t) => t == 0 ? 1 : Math.Sqrt(prev) / t)
    .Run(timeStep, timeLimit, Method.OutterForce);

var thirdScatter1 = new Scatter
{   
    x = time,
    y = third1,
    marker = new Marker { color = "blue" },
    line = new Line { shape = "spline" }
};

var thirdScatter2 = new Scatter
{   
    x = time,
    y = third2,
    marker = new Marker { color = "green" },
    line = new Line { shape = "spline" }
};

var thirdScatter3 = new Scatter
{   
    x = time,
    y = third3,
    marker = new Marker { color = "red" },
    line = new Line { shape = "spline" }
};

Chart.Plot(new List<Trace>() { thirdScatter1, thirdScatter2, thirdScatter3 }, layout)

<!DOCTYPE html>